In [1]:
import numpy as np
import tensorflow as tf

# 07-03: Convolution Operation Naive Backward Pass

Try calculating NumPy, then check with TensorFlow and numerical gradient

#### 1. Basic
$(4 \times 4) * (3 \times 3) = (2 \times 2)$

#### 2. Padding
$(4 \times 4) * (3 \times 3) = (4 \times 4)$ where $P=1$

#### 3. Stride
$(7 \times 7) * (3 \times 3) = (3 \times 3)$ where $S=2$

#### 4. Padding and Stride
$(7 \times 7) * (3 \times 3) = (4 \times 4)$ where $P=1, S=2$

#### 5. Channel
$(4 \times 4 \times 3) * (3 \times 3 \times 3) = (2 \times 2)$

#### 6. Channel and bias 
$(4 \times 4 \times 3) * (3 \times 3 \times 3) + (1) = (2 \times 2)$

#### 7. Multiple Filters
$(4 \times 4 \times 3) * (3 \times 3 \times 3 \times 4) = (2 \times 2 \times 4)$

#### 8.Multiple Filters + bias 
$(4 \times 4 \times 3) * (3 \times 3 \times 3 \times 4) + (4)= (2 \times 2 \times 4)$

#### 9. Mini-batch + bias
$(3 \times 4 \times 4 \times 3) * (3 \times 3 \times 3 \times 4) + (4)= (3 \times 2 \times 2 \times 4)$

#### 10. RGB Mini-batch $*$ Multiple Filters with stride and padding
$(3 \times 7 \times 7 \times 3) * (3 \times 3 \times 3 \times 4) + (4)= (3 \times 4 \times 4 \times 4)$ where $P=1, S=2$


In [27]:
def float_sequence(size):
    return np.arange(size, dtype=np.float32)

### 1. Basic Convolution

$(4 \times 4) * (3 \times 3) = (2 \times 2)$

### 2. Convolution with padding

$(4 \times 4) * (3 \times 3) = (4 \times 4)$ where $P=1$

### 3. Convolution with Stride

$(7 \times 7) * (3 \times 3) = (3 \times 3)$ where $S=2$

### 4. Padding and Stride

$(7 \times 7) * (3 \times 3) = (4 \times 4)$ where $P=1, S=2$

### 5. Channel

$(4 \times 4 \times 3) * (3 \times 3 \times 3) = (2 \times 2)$

### 6. Channel and bias 

$(4 \times 4 \times 3) * (3 \times 3 \times 3) + (1) = (2 \times 2)$

### 7. Multiple Filters

$(4 \times 4 \times 3) * (3 \times 3 \times 3 \times 4) = (2 \times 2 \times 4)$

### 8. Multiple Filters + bias 

$(4 \times 4 \times 3) * (3 \times 3 \times 3 \times 4) + (4)= (2 \times 2 \times 4)$

### 9. Mini-batch + bias

$(3 \times 4 \times 4 \times 3) * (3 \times 3 \times 3 \times 4) + (4)= (3 \times 2 \times 2 \times 4)$

#### 10. RGB Mini-batch $*$ Multiple Filters with stride and padding
$(3 \times 7 \times 7 \times 3) * (3 \times 3 \times 3 \times 4) + (4)= (3 \times 4 \times 4 \times 4)$ where $P=1, S=2$